<h1 align=center><font size = 5>Machine Learning models for aiding the decision-making process in emergency departments</font></h1>

<h1>Tabla comparativa de algoritmos<h1>
<h3>Descripción</h3>
<p>
En este script se desarrollará una serie de predicciones basadas en datos del hospital San Juan de Dios Curicó, correspondientes al año 2018 representados por registros de urgencias. El objetivo es predecir, mediante un conjunto de algoritmos, la necesidad de hospitalización de un paciente de urgencias,tomando como input, datos proporcionados por el paciente en la etapa de registro, sus signos vitales registrados en la etapa de triage y el diagnóstico ofrecido por el médico tratante.
Se correrán algoritmos de prodicción tales como árboles y bosques de desición, regresión logística, support vector machines y redes neuronales. Para finalmente evaluar el rendimiento de cada algoritmo en términos de la predicción, mediante indicadores tales como Acuraccy, F1-Score, Curva ROC, Índice de Jaccard y logloss
</p>

<h1 id="Descripción de datos">Descripción de datos</h1>
<p>
Los datos utilizados fueros proporcionados por el Hospital San Juan de Dios, Curicó, Chile y corresponden a 4.971 registros de pacientes que asistieron a urgencias durante el periodo comprendido entre el 1 de enero de 2018 y agosto de 2019, los datos fueron limpiados y transformados en un script desarrollado previamente
<ul>
    <li>Datos: <a href="https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD" target="_blank">https://drive.google.com/open?id=1Bp7_MnK6cGwgBuwIq1a8S4DS_0wVDiAD</a></li>
    <li>Tipo de datos: csv</li>
   </ul>
<p>
Las variables presentes en los datos se describen a continuación:
<ul>    
   
   <li><b>PAC_EDAD</b>: corresponde a la edad del paciente en enteros</li>
   <li><b>MOTIVO_CONSULTA</b>: corresponde a la razón por la que el paciente acude al servicio de urgencias string</li>
   <li><b>MEDIO</b>: corresponde al medio de llegada, mediante el que el paciente acude al servicio de urgencias</li>
   <li><b>SEXO</b>: corresponde al sexo del paciente</li>
   <li><b>CAT</b>: corresponde a la categoría de gravedad asignada al paciente en el proceso de Triage</li>
   <li><b>PRESION_SIST</b>: corresponde la presión sistólica del paciente </li>
   <li><b>PRESION_DIAST</b>: corresponde la presióndiastólica del paciente</li>
   <li><b>SATO2</b>: Dato numérico que representa la saturometria del paciente (Nivel de oxigeno en la sangre)</li>
   <li><b>TEMPERATURA</b>: corresponde a la temperatura corporal del paciente en el momento de la categorización</li>
   <li><b>GLASGOW</b>: corresponde a al nivel registrado por el paciente en la escala Glasgow</li>
   <li><b>DM</b>: corresponde si el paciente presenta o no Diabetes Mellitus</li>
   <li><b>EVA</b>: corresponde si se aplica al paciente una evaluación de vias aéreas</li>
   <li><b>HGT</b>: corresponde a la medida de azucar en la sangre del paciente</li>
   <li><b>LCFA</b>: corresponde a si el paciente presenta obstrucción crónica de vías aéreas</li>
   <li><b>FR</b>: corresponde a la frecuencia respiratoria del paciente</li>
   <li><b>HTA</b>: corresponde a si el paciente posee Hipertención Arterial</li>
   <li><b>HORA_INSC</b>: corresponde a la hora en la que el paciente fue categorizado</li>
   <li><b>MIN_INSC</b>: corresponde al minuto en que el paciente fue categorizado</li>
   <li><b>TIEMPO_ESPERA_CAT</b>: corresponde al tiempo que espera el paciente luego de ser registrado, para ser categorizado</li>
      
</ul>
</p>

Cargar paquetes necesarios

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn import metrics
import sklearn as sk  
#import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import f1_score
#from sklearn.metrics import jaccard_similarity_score
from sklearn.metrics import log_loss
from sklearn.metrics import f1_score
#Métodos de tuneo de parámetros
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

Lectura de datos

In [ ]:
archivo= 'df_definitiva_B.csv'
df_hosp= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [ ]:
archivo='df_cat_soloTexto.csv'
df_cat= pd.read_csv(archivo,encoding='latin-1',sep=",")

In [ ]:
df_hosp =df_hosp[['ID_PACIENTE','DESTINO']]

In [ ]:
df_cat.columns
df_cat.drop(columns =['DESC_EVENTO','CAT'], inplace = True)

In [ ]:
df_hosp=df_hosp[['ID_PACIENTE', 'PAC_EDAD', 'SATO2', 'TEMPERATURA',
       'GLASGOW', 'EVA', 'HGT', 'FR', 'DESTINO', 'SEXO_M',
       'SEXO_F', 'DM_D', 'DM_N', 'DM_S', 'LCFA_D', 'LCFA_N', 'LCFA_S',
       'LCFA_D.1', 'LCFA_N.1', 'LCFA_S.1']]

In [ ]:
#df_hosp = pd.merge(df_hosp,df_cat, on='ID_PACIENTE')

In [ ]:
df_hosp.drop(columns =['Unnamed: 0'], inplace = True)
df_cat.drop(columns =['Unnamed: 0'], inplace = True)

Unión de bases de datos

In [ ]:
df_hosp =  df_hosp.join(df_cat, lsuffix='_caller', rsuffix='_other')

In [ ]:
# Limpiando de faltantes las columnas
df_hosp.dropna(subset=["ID_PACIENTE_caller"], axis=0, inplace = True)
df_hosp.dropna(subset=["ID_PACIENTE_other"], axis=0, inplace = True)

In [ ]:
df_hosp.shape

In [ ]:
# Limpiando de faltantes las columnas
df_hosp.dropna(subset=["DESTINO"], axis=0, inplace = True)

In [ ]:
#Quitando ID_PACIENTE
df_hosp.drop(columns =["ID_PACIENTE_caller"], inplace = True)
df_hosp.drop(columns =["ID_PACIENTE_other"], inplace = True)

In [ ]:
y =df_hosp['DESTINO']

In [ ]:
#Quitando 'DESTINO'
#df_hosp.drop(columns =['DESTINO'], inplace = True)
#df_hosp.drop(columns =['CAT'], inplace = True)
df_hosp.drop(columns =['DESTINO'], inplace = True)

In [ ]:
missing_data = df_hosp.isnull()

    
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("") 

Seleccionar las variables relevantes para la predicción y asignarselas a la matriz X, de variables independientes

In [ ]:
X=df_hosp
X.shape

In [ ]:
y.shape

Preprocesamiento Sexo

In [ ]:
le_sexo = preprocessing.LabelEncoder()
le_sexo.fit([ 'FEMENINO','MASCULINO'])
X[:,3] = le_sexo.transform(X[:,3])

Preprocesamiento DM (Diabetes Mellitus)

In [ ]:
le_DM = preprocessing.LabelEncoder()
le_DM.fit([ 'S', 'N','D'])
X[:,5] = le_DM.transform(X[:,5])

Preprocesamiento LCFA (Limintación crónica del flujo aéreo)

In [ ]:
le_LCFA = preprocessing.LabelEncoder()
le_LCFA.fit([ 'S', 'N','D'])
X[:,8] = le_LCFA.transform(X[:,8]) 

Preprocesamiento HTA (Hipertención Arterial)

In [ ]:
le_HTA = preprocessing.LabelEncoder()
le_HTA.fit([ 'S', 'N','D'])
X[:,10] = le_HTA.transform(X[:,10]) 

Preprocesamiento CAT ( Categoría)

In [ ]:
le_CAT = preprocessing.LabelEncoder()
le_CAT.fit([ 'C1', 'C2','C3','C4','C5'])
X[:,11] = le_CAT.transform(X[:,11]) 

In [ ]:
le_diag = preprocessing.LabelEncoder()
le_diag.fit(["ABDOMEN AGUDO",
"ABSCESO ANAL",
"ABSCESO ANORRECTAL",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DE GLUTEOS",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DE LA CARA",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DE MIEMBRO",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DE OTROS SITIOS",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DE SITIO NO ESPECIFICADO",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DEL CUELLO",
"ABSCESO CUTANEO, FURUNCULO Y CARBUNCO DEL TRONCO",
"ABSCESO DEL HÃGADO",
"ABSCESO DEL PULMÃ“N CON NEUMONIA",
"ABSCESO PERIAMIGDALINO",
"ABSCESO RENAL Y PERIRRENAL",
"ABSCESO Y QUISTE SUBCUTÃNEO FEOMICOTICO",
"ACCIDENTE DE TRANSPORTE NO ESPECIFICADO",
"ACCIDENTE VASCULAR ENCEFALICO AGUDO, NO ESPECIFICADO COMO HEMORRAGICO O ISQUEMICO",
"ACIDOSIS",
"ADHERENCIAS (BRIDAS) INTESTINALES CON OBSTRUCCION",
"AGRESION CON DROGAS, MEDICAMENTOS Y SUSTANCIAS BIOLOGICAS, OCURRIDA EN UNA INSTITUCION RESIDENCIAL",
"AGRESION POR AHOGAMIENTO Y SUMERSION, OCURRIDA EN UN AREA DE DEPORTE O ATLETISMO",
"ALCOHOLISMO, NIVEL DE INTOXICACION NO ESPECIFICADO",
"ALERGIA NO ESPECIFICADA",
"ALETEO AURICULAR TÃPICO",
"ALTERACION VISUAL, NO ESPECIFICADA",
"ALTERACIONES DE LA VISION EN ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"ALUCINACIONES, NO ESPECIFICADAS",
"AMAUROSIS FUGAZ",
"AMIGDALITIS AGUDA DEBIDA A OTROS MICROORGANISMOS ESPECIFICADOS",
"AMIGDALITIS AGUDA, NO ESPECIFICADA",
"AMPUTACION DE MIEMBRO(S), COMO CAUSA DE REACCION ANORMAL DEL PACIENTE O DE COMPLICACION POSTERIOR, SIN MENCION DE INCIDENTE EN EL MOMENTO DE EFECTUAR EL PROCEDIMIENTO",
"AMPUTACION DEL PIE, NIVEL NO ESPECIFICADO",
"AMPUTACION TRAUMATICA DE LA MUÃ‘ECA Y DE LA MANO, NIVEL NO ESPECIFICADO",
"AMPUTACION TRAUMATICA DE OTRO DEDO UNICO (COMPLETA) (PARCIAL)",
"AMPUTACION TRAUMATICA DEL PULGAR (COMPLETA) (PARCIAL)",
"ANEMIA DE TIPO NO ESPECIFICADO",
"ANEMIA EN OTRAS ENFERMEDADES CRONICAS CLASIFICADAS EN OTRA PARTE",
"ANEMIA HEMOLÃTICA ADQUIRIDA, SIN OTRA ESPECIFICACIÃ“N",
"ANEMIA POR DEFICIENCIA DE HIERRO SIN OTRA ESPECIFICACIÃ“N",
"ANEMIA POSTHEMORRAGICA AGUDA",
"ANEURISMA DE LA AORTA ABDOMINAL, SIN MENCION DE RUPTURA",
"ANEURISMA DE LA AORTA TORACOABDOMINAL, SIN MENCION DE RUPTURA",
"ANGINA DE PECHO CON ESPASMO DOCUMENTADO",
"ANGINA DE PECHO, NO ESPECIFICADA",
"ANGINA INESTABLE",
"ANURIA Y OLIGURIA",
"APENDICITIS AGUDA CON PERITONITIS GENERALIZADA",
"APENDICITIS AGUDA CON PERITONITIS LOCALIZADA",
"APENDICITIS, NO ESPECIFICADA",
"ARRITMIA CARDIACA, NO ESPECIFICADA",
"ARTRITIS EN OTRAS ENFERMEDADES BACTERIANAS CLASIFICADAS EN OTRA PARTE",
"ARTRITIS NO ESPECIFICADA",
"ARTRITIS PIÃ“GENA, NO ESPECIFICADA",
"ARTRITIS REUMATOIDE, NO ESPECIFICADA",
"ARTROPATÃA REACTIVA NO ESPECIFICADA",
"ARTROSIS POSTRAUMATICA DE LA PRIMERA ARTICULACION CARPOMETACARPIANA, BILATERAL",
"ARTROSIS PRIMARIA DE OTRAS ARTICULACIONES",
"ARTROSIS SECUNDARIA MULTIPLE",
"ARTROSIS, NO ESPECIFICADA",
"ASCITIS",
"ASISTENCIA Y AJUSTE DE DISPOSITIVOS DE ACCESO VASCULAR",
"ASMA, NO ESPECIFICADO",
"ATENCIÃ“N DE COLOSTOMIA",
"ATENCIÃ“N DE GASTROSTOMÃA",
"ATENCIÃ“N DE TRAQUEOSTOMIA",
"ATEROSCLEROSIS DE LAS ARTERIAS DE LOS MIEMBROS",
"BALANITIS EN ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"BALANOPOSTITIS",
"BLEFAROPTOSIS",
"BLOQUEO AURICULOVENTRICULAR COMPLETO",
"BLOQUEO AURICULOVENTRICULAR DE SEGUNDO GRADO",
"BLOQUEO BIFASCICULAR",
"BOCIO MULTINODULAR NO TOXICO",
"BRADICARDIA, NO ESPECIFICADA",
"BRONCONEUMONIA, NO ESPECIFICADA",
"BRONQUITIS AGUDA DEBIDA A ESTREPTOCOCOS",
"BRONQUITIS AGUDA DEBIDA A OTROS MICROORGANISMOS ESPECIFICADOS",
"BRONQUITIS AGUDA, NO ESPECIFICADO",
"BRONQUITIS CRONICA NO ESPECIFICADA",
"BRONQUITIS, NO ESPECIFICADA COMO AGUDA O CRONICA",
"BURSITIS DE LA MANO",
"BURSITIS DEL HOMBRO",
"CADERA INESTABLE",
"CAIDA EN EL MISMO NIVEL POR DESLIZAMIENTO, TROPEZON O TRASPIE, OCURRIDA EN OTRO LUGAR ESPECIFICADO",
"CAIDA NO ESPECIFICADA, OCURRIDA EN UNA VIVIENDA",
"CALCULO DE CONDUCTO BILIAR CON COLANGITIS",
"CALCULO DE CONDUCTO BILIAR CON COLECISTITIS",
"CALCULO DE CONDUCTO BILIAR SIN COLANGITIS NI COLECISTITIS",
"CALCULO DE LA VESICULA BILIAR CON COLECISTITIS AGUDA",
"CALCULO DE LA VESICULA BILIAR CON OTRA COLECISTITIS",
"CALCULO DE LA VESICULA BILIAR SIN COLECISTITIS",
"CALCULO DEL RIÃ‘Ã“N CON CALCULO DEL URETER",
"CALCULO DEL URETER",
"CALCULO URINARIO, NO ESPECIFICADO",
"CARCINOMA DE VÃAS BILIARES INTRAHEPATICAS",
"CARCINOMA IN SITU DE LA MAMA, PARTE NO ESPECIFICADA",
"CARCINOMA IN SITU DE OTRAS PARTES ESPECIFICADAS DE Ã“RGANOS DIGESTIVOS",
"CARCINOMA IN SITU DE OTROS Ã“RGANOS URINARIOS Y DE LOS NO ESPECIFICADOS",
"CARCINOMA IN SITU DE OTROS SITIOS DE Ã“RGANOS GENITALES FEMENINOS Y DE LOS NO ESPECIFICADOS",
"CARDIOMIOPATIA DILATADA",
"CATARATA EN OTRAS ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"CATARATA, NO ESPECIFICADA",
"CEFALEA DEBIDA A TENSION",
"CEFALEA POSTRAUMATICA CRONICA",
"CEFALEA VASCULAR, NO CLASIFICADA EN OTRA PARTE",
"CEFALEA",
"CEGUERA MONOCULAR",
"CELULITIS DE LA CARA",
"CELULITIS DE LOS DEDOS DE LA MANO Y DEL PIE",
"CELULITIS DE OTRAS PARTES DE LOS MIEMBROS",
"CELULITIS DE OTROS SITIOS",
"CELULITIS DE SITIO NO ESPECIFICADO",
"CELULITIS DEL TRONCO",
"CERUMEN IMPACTADO",
"CERVICALGIA",
"CHASQUIDO DE LA CADERA",
"CHOQUE SÃ‰PTICO",
"CIRROSIS BILIAR SECUNDARIA",
"CIRROSIS BILIAR, NO ESPECIFICADA",
"CIRROSIS HEPATICA ALCOHOLICA",
"CISTICERCOSIS DEL SISTEMA NERVIOSO CENTRAL",
"CISTITIS AGUDA",
"CISTITIS, NO ESPECIFICADA",
"COLANGITIS",
"COLECISTITIS AGUDA",
"COLECISTITIS, NO ESPECIFICADA",
"COLICO RENAL, NO ESPECIFICADO",
"COLITIS DEL LADO IZQUIERDO",
"COLITIS INDETERMINADA",
"COLITIS ULCERATIVA, SIN OTRA ESPECIFICACION",
"COLITIS Y GASTROENTERITIS ALERGICAS Y DIETETICAS",
"COLITIS Y GASTROENTERITIS DEBIDAS A RADIACION",
"COLITIS Y GASTROENTERITIS NO INFECCIOSAS, NO ESPECIFICADAS",
"COMPLICACION DE PROCEDIMIENTOS, NO ESPECIFICADA",
"COMPLICACION MECANICA DE CATETER PARA DIÃLISIS VASCULAR",
"COMPLICACION MECANICA DE CATETER URINARIO (FIJO)",
"CONCUSION Y EDEMA DE LA MEDULA ESPINAL LUMBAR",
"CONGESTION Y HEMORRAGIA DE LA PROSTATA",
"CONJUNTIVITIS AGUDA, NO ESPECIFICADA",
"CONJUNTIVITIS MUCOPURULENTA",
"CONJUNTIVITIS, NO ESPECIFICADA",
"CONSTIPACION",
"CONSULTA, NO ESPECIFICADA",
"CONTRACTURA MUSCULAR",
"CONTUSION DE DEDO(S) DE LA MANO CON DAÃ‘O DE LA(S) UÃ‘A(S)",
"CONTUSION DE DEDO(S) DE LA MANO, SIN DAÃ‘O DE LA(S) UÃ‘A(S)",
"CONTUSION DE DEDO(S) DEL PIE SIN DAÃ‘O DE LA(S) UÃ‘A(S)",
"CONTUSION DE LA CADERA",
"CONTUSION DE LA GARGANTA",
"CONTUSION DE LA REGION LUMBOSACRA Y DE LA PELVIS",
"CONTUSION DE LA RODILLA",
"CONTUSION DE LOS PARPADOS Y DE LA REGION PERIOCULAR",
"CONTUSION DE OTRAS PARTES DE LA MUÃ‘ECA Y DE LA MANO",
"CONTUSION DE OTRAS PARTES DEL ANTEBRAZO Y DE LAS NO ESPECIFICADAS",
"CONTUSION DE OTRAS PARTES Y DE LAS NO ESPECIFICADAS DE LA PIERNA",
"CONTUSION DEL CODO",
"CONTUSION DEL GLOBO OCULAR Y DEL TEJIDO ORBITARIO",
"CONTUSION DEL HOMBRO Y DEL BRAZO",
"CONTUSION DEL MUSLO",
"CONTUSION DEL TOBILLO",
"CONTUSION DEL TORAX",
"CORROSION DE LA CABEZA Y DEL CUELLO, DE PRIMER GRADO",
"CORROSION DEL TOBILLO Y DEL PIE, DE SEGUNDO GRADO",
"COXARTROSIS, NO ESPECIFICADA",
"CUERPO EXTRAÃ‘O EN EL ESOFAGO",
"CUERPO EXTRAÃ‘O EN EL SACO CONJUNTIVAL",
"CUERPO EXTRAÃ‘O EN LA CORNEA",
"CUERPO EXTRAÃ‘O EN PARTE EXTERNA DEL OJO, SITIO NO ESPECIFICADO",
"CUIDADO POSTERIOR A LA CIRUGÃA, NO ESPECIFICADO",
"DEFECTOS CUALITATIVOS DE LAS PLAQUETAS",
"DEFICIENCIA DE VITAMINA B, NO ESPECIFICADA",
"DEGENERACION CEREBRAL SENIL NO CLASIFICADA EN OTRA PARTE",
"DELIRIO, NO ESPECIFICADO",
"DEMENCIA EN LA ENFERMEDAD DE ALZHEIMER, NO ESPECIFICADA (G30.9+)",
"DEMENCIA EN OTRAS ENFERMEDADES ESPECIFICADAS CLASIF EN OTRA PARTE",
"DEMENCIA, NO ESPECIFICADA",
"DERMATITIS ALERGICA DE CONTACTO DEBIDA A DROGAS EN CONTACTO CON LA PIEL",
"DERMATITIS ALERGICA DE CONTACTO, DE CAUSA NO ESPECIFICADA",
"DERMATITIS INFECCIOSA",
"DERMATITIS, NO ESPECIFICADAS",
"DERRAME PLEURAL EN AFECCIONES CLASIFICADAS EN OTRA PARTE",
"DERRAME PLEURAL NO CLASIFICADO EN OTRA PARTE",
"DESGARRO DE HERIDA OPERATORIA, NO CLASIFICADO EN OTRA PARTE",
"DESNUTRICIÃ“N PROTEICOCALORICA SEVERA, NO ESPECIFICADA",
"DESORIENTACIÃ“N NO ESPECIFICADA",
"DESPRENDIMIENTO DE LA RETINA POR TRACCION",
"DESPRENDIMIENTO SEROSO DE LA RETINA",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON CETOACIDOSIS",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON COMPLICACIONES CIRCULATORIAS PERIFÃ‰RICAS",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON COMPLICACIONES MÃšLTIPLES",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON COMPLICACIONES NEUROLÃ“GICAS",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON COMPLICACIONES NO ESPECIFICADAS",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON COMPLICACIONES RENALES",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, CON OTRAS COMPLICACIONES ESPECIFICADAS",
"DIABETES MELLITUS ASOCIADA CON DESNUTRICIÃ“N, SIN MENCIÃ“N DE COMPLICACIÃ“N",
"DIABETES MELLITUS INSULINODEPENDIENTE CON CETOACIDOSIS",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES CIRCULATORIAS PERIFÃ‰RICAS",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES MÃšLTIPLES",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES NEUROLÃ“GICAS",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES NO ESPECIFICADAS",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES OFTÃLMICAS",
"DIABETES MELLITUS INSULINODEPENDIENTE CON COMPLICACIONES RENALES",
"DIABETES MELLITUS INSULINODEPENDIENTE CON OTRAS COMPLICACIONES ESPECIFICADAS",
"DIABETES MELLITUS INSULINODEPENDIENTE SIN MENCIÃ“N DE COMPLICACIÃ“N",
"DIABETES MELLITUS NO ESPECIFICADA, CON CETOACIDOSIS",
"DIABETES MELLITUS NO ESPECIFICADA, CON COMA",
"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLICACIONES CIRCULATORIAS PERIFÃ‰RICAS",
"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLICACIONES MÃšLTIPLES",
"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLICACIONES NEUROLÃ“GICAS",
"DIABETES MELLITUS NO ESPECIFICADA, CON COMPLICACIONES NO ESPECIFICADAS",
"DIABETES MELLITUS NO ESPECIFICADA, CON OTRAS COMPLICACIONES ESPECIFICADAS",
"DIABETES MELLITUS NO ESPECIFICADA, SIN MENCIÃ“N DE COMPLICACIÃ“N",
"DIABETES MELLITUS NO INSULINODEPENDIENTE CON COMPLICACIONES CIRCULATORIAS PERIFÃ‰RICAS",
"DIABETES MELLITUS NO INSULINODEPENDIENTE CON COMPLICACIONES NO ESPECIFICADAS",
"DIABETES MELLITUS NO INSULINODEPENDIENTE CON COMPLICACIONES RENALES",
"DIABETES MELLITUS NO INSULINODEPENDIENTE CON OTRAS COMPLICACIONES ESPECIFICADAS",
"DIABETES MELLITUS NO INSULINODEPENDIENTE SIN MENCIÃ“N DE COMPLICACIÃ“N",
"DIARREA FUNCIONAL",
"DIPLOPIA",
"DISARTRIA Y ANARTRIA",
"DISECCION DE AORTA (CUALQUIER PARTE)",
"DISFAGIA",
"DISFONIA",
"DISMENORREA, NO ESPECIFICADA",
"DISNEA",
"DISPEPSIA",
"DISURIA",
"DOLOR ABDOMINAL LOCALIZADO EN PARTE SUPERIOR",
"DOLOR AGUDO",
"DOLOR EN ARTICULACIÃ“N",
"DOLOR EN EL PECHO AL RESPIRAR",
"DOLOR EN EL PECHO, NO ESPECIFICADO",
"DOLOR EN LA COLUMNA DORSAL",
"DOLOR EN MIEMBRO",
"DOLOR LOCALIZADO EN OTRAS PARTES INFERIORES DEL ABDOMEN",
"DOLOR OCULAR",
"DOLOR PRECORDIAL",
"DOLOR, NO ESPECIFICADO",
"DORSALGIA, NO ESPECIFICADA",
"DUODENITIS",
"EDEMA ANGIONEUROTICO",
"EDEMA CEREBRAL",
"EDEMA GENERALIZADO",
"EDEMA LOCALIZADO",
"EDEMA PULMÃ“NAR",
"EDEMA, NO ESPECIFICADO",
"EFECTO ADVERSO NO ESPECIFICADO DE DROGA O MEDICAMENTO",
"EFECTO TOXICO DE ALCOHOL, NO ESPECIFICADO",
"EFECTO TOXICO DE CETONAS",
"EFECTO TOXICO DE OTRAS SUSTANCIAS INORGANICAS, ESPECIFICADAS",
"EFECTO TOXICO DE SUSTANCIA NO ESPECIFICADA",
"EFECTO TOXICO DEL MONOXIDO DE CARBONO",
"EFECTOS ADVERSOS DE ANTAGONISTAS DE ANTICOAGULANTES, DE VITAMINA K Y DE OTROS COAGULANTES",
"EFECTOS ADVERSOS DE ANTICOAGULANTES",
"EFECTOS ADVERSOS DE BENZODIACEPINAS",
"EFECTOS ADVERSOS DE DROGAS ANALGESICAS, ANTIPIRETICAS Y ANTIINFLAMATORIAS NO ESPECIFICADAS",
"EFECTOS ADVERSOS DE DROGAS O MEDICAMENTOS NO ESPECIFICADOS",
"EFECTOS ADVERSOS DE LOS DERIVADOS DE LA BENZOTIADIAZINA",
"EFECTOS ADVERSOS DE OTRAS DROGAS Y MEDICAMENTOS",
"EMBOLIA PULMÃ“NAR CON MENCION DE CORAZÃ“N PULMÃ“NAR AGUDO",
"EMBOLIA PULMÃ“NAR SIN MENCION DE CORAZÃ“N PULMÃ“NAR AGUDO",
"EMBOLIA Y TROMBOSIS DE ARTERIA NO ESPECIFICADA",
"EMBOLIA Y TROMBOSIS DE ARTERIAS DE LOS MIEMBROS INFERIORES",
"EMBOLIA Y TROMBOSIS DE ARTERIAS DE LOS MIEMBROS SUPERIORES",
"EMBOLIA Y TROMBOSIS DE ARTERIAS DE LOS MIEMBROS, NO ESPECIFICADAS",
"EMBOLIA Y TROMBOSIS DE OTRAS ARTERIAS",
"EMBOLIA Y TROMBOSIS DE OTRAS VENAS ESPECIFICADAS",
"EMBOLIA Y TROMBOSIS DE VENA NO ESPECIFICADA",
"ENCEFALOPATIA NO ESPECIFICADA",
"ENCEFALOPATIA TOXICA",
"ENCELOFALOPATIA DE WERNICKE",
"ENDOMETRIOSIS, NO ESPECIFICADA",
"ENFERMEDAD CARDIACA HIPERTENSIVA CON INSUFICIENCIA CARDIACA (CONGESTIVA)",
"ENFERMEDAD CARDIACA HIPERTENSIVA SIN INSUFICIENCIA CARDIACA (CONGESTIVA)",
"ENFERMEDAD CARDIORRENAL HIPERTENSIVA CON INSUFICIENCIA RENAL",
"ENFERMEDAD CARDIORRENAL HIPERTENSIVA CON INSUFICIENCIAS CARDIACA Y RENAL (CONGESTIVA)",
"ENFERMEDAD CEREBROVASCULAR, NO ESPECIFICADA",
"ENFERMEDAD DE ALZHEIMER, NO ESPECIFICADA",
"ENFERMEDAD DE LA VESICULA BILIAR, NO ESPECIFICADA",
"ENFERMEDAD DE LAS VÃAS BILIARES, NO ESPECIFICADA",
"ENFERMEDAD DE LOS VASOS PULMÃ“NARES, NO ESPECIFICADO",
"ENFERMEDAD DE MENIERE",
"ENFERMEDAD DE PARKINSON",
"ENFERMEDAD DEL ESTOMAGO Y DEL DUODENO, NO ESPECIFICADA",
"ENFERMEDAD DEL REFLUJO GASTROESOFAGICO CON ESOFAGITIS",
"ENFERMEDAD DEL REFLUJO GASTROESOFAGICO SIN ESOFAGITIS",
"ENFERMEDAD DEL SISTEMA DIGESTIVO, NO ESPECIFICADA",
"ENFERMEDAD DIVERTICULAR DEL INTESTINO GRUESO SIN PERFORACION NI ABSCESO",
"ENFERMEDAD DIVERTICULAR DEL INTESTINO, PARTE NO ESPECIFICADA, SIN PERFORACION NI ABSCESO",
"ENFERMEDAD HEPATICA ALCOHOLICA, NO ESPECIFICADA",
"ENFERMEDAD ISQUEMICA AGUDA DEL CORAZÃ“N, NO ESPECIFICADA",
"ENFERMEDAD POR VIRUS DE LA INMUNODEFICIENCIA HUMANA (VIH), SIN OTRA ESPECIFICACIÃ“N",
"ENFERMEDAD PULMÃ“NAR INTERSTICIAL, NO ESPECIFICADA",
"ENFERMEDAD PULMÃ“NAR OBSTRUCTIVA CRONICA C/INFECC. AG.DE VÃAS RESPIRATORIAS INFERIORES",
"ENFERMEDAD PULMÃ“NAR OBSTRUCTIVA CRONICA CON EXACERBACION AGUDA, NO ESPECIFICADA",
"ENFERMEDAD PULMÃ“NAR OBSTRUCTIVA CRONICA, NO ESPECIFICADA",
"ENFERMEDAD RENAL CRÃ“NICA, ESTADIO 3",
"ENFERMEDAD RENAL CRÃ“NICA, ESTADIO 4",
"ENFERMEDAD RENAL CRÃ“NICA, ESTADIO 5",
"ENFERMEDAD RENAL CRÃ“NICA, NO ESPECIFICADA",
"ENFERMEDAD RENAL HIPERTENSIVA CON INSUFICIENCIA RENAL",
"ENFERMEDAD TOXICA DEL HÃGADO CON OTROS TRASTORNOS HEPATICOS",
"ENFERMEDAD VASCULAR PERIFERICA, NO ESPECIFICADA",
"ENFISEMA, NO ESPECIFICADO",
"ENTEROCOLITIS DEBIDA A CLOSTRIDIUM DIFFICILE",
"ENVENENAMIENTO AUTOINFLIGIDO INTENCIONALMENTE POR, Y EXPOSICION A DROGAS ANTIEPILEPTICAS, SEDANTES, HIPNOTICAS, ANTI PARKINSONIANAS Y PSICOTROPICAS, NO CLASIFICADAS EN OTRA PARTE, OCURRIDOS EN OTRO LUGAR ESPECIFICADO",
"ENVENENAMIENTO POR BENZODIAZEPINAS",
"EPILEPSIA Y SINDROMES EPILEPTICOS IDIOPATICOS GENERALIZADOS",
"EPILEPSIA, TIPO NO ESPECIFICADO",
"EPISODIO DEPRESIVO GRAVE CON SINTOMAS PSICÃ“TICOS",
"EPISODIO DEPRESIVO GRAVE SIN SINTOMAS PSICÃ“TICOS",
"EPISODIO DEPRESIVO LEVE",
"EPISODIO DEPRESIVO MODERADO",
"EPISODIO DEPRESIVO, NO ESPECIFICADO",
"EPISODIO MANIACO, NO ESPECIFICADO",
"EPISTAXIS",
"EQUIMOSIS ESPONTANEA",
"ERISIPELA",
"ERITEMA NUDOSO",
"ESCABIOSIS",
"ESCLEROSIS HEPATICA",
"ESGUINCES Y TORCED. QUE COMPROMETEN LIGAM.LATERALES (EXTERNO) (INTERNO) DE LA RODILLA",
"ESGUINCES Y TORCEDURAS DE DEDO(S) DE LA MANO",
"ESGUINCES Y TORCEDURAS DE LA ARTICULACION ACROMIOCLAVICULAR",
"ESGUINCES Y TORCEDURAS DE LA COLUMNA CERVICAL",
"ESGUINCES Y TORCEDURAS DE LA COLUMNA LUMBAR",
"ESGUINCES Y TORCEDURAS DEL TOBILLO",
"ESOFAGITIS",
"ESPLENOMEGALIA, NO CLASIFICADA EN OTRA PARTE",
"ESPONDILOSIS NO ESPECIFICADA",
"ESQUIZOFRENIA, NO ESPECIFICADA",
"ESTADO ASMATICO",
"ESTADO DE ABSTINENCIA DE ALCOHOL",
"ESTADO DE ABSTINENCIA DE ALCOHOL, CON DELIRIO",
"ESTADO MIGRAÃ‘OSO",
"ESTADOS MENOPAUSICOS Y CLIMATERICOS FEMENINOS",
"EXAMEN DE LABORATORIO",
"EXAMEN DE PESQUISA ESPECIAL PARA DIABETES MELLITUS",
"EXAMEN DE PESQUISA ESPECIAL PARA TUMOR DE SITIO NO ESPECIFICADO",
"EXAMEN DE PESQUISA ESPECIAL PARA TUMORES DE OTROS SITIOS",
"EXAMEN DE PESQUISA ESPECIAL, NO ESPECIFICADO",
"EXAMEN MEDICO GENERAL",
"EXAMEN PARA FINES ADMINISTRATIVOS, NO ESPECIFICADO",
"EXAMEN Y OBSERVACION CONSECUTIVOS A ACCIDENTE DE TRANSPORTE",
"FALLA Y RECHAZO DE TRASPLANTE DE HÃGADO",
"FARINGITIS AGUDA, NO ESPECIFICADA",
"FASCITIS NECROTIZANTE",
"FIBRILACIÃ“N AURICULAR CRÃ“NICA",
"FIBRILACIÃ“N AURICULAR PAROXÃ“STICA",
"FIBRILACIÃ“N AURICULAR PERSISTENTE",
"FIBRILACIÃ“N Y ALETEO AURICULAR, NO ESPECIFICADO",
"FIBRILACION Y ALETEO VENTRICULAR",
"FIBROMIALGIA",
"FIEBRE, NO ESPECIFICADA",
"FISTULA ANORRECTAL",
"FISTULA ARTERIOVENOSA, ADQUIRIDA",
"FISTULA DEL TRACTO GENITAL FEMENINO, SIN OTRA ESPECIFICACION",
"FISURA ANAL AGUDA",
"FISURA ANAL, NO ESPECIFICADA",
"FLEBITIS Y TROMBOFLEBITIS DE LA VENA FEMORAL",
"FLEBITIS Y TROMBOFLEBITIS DE LOS MIEMBROS INFERIORES, NO ESPECIFICADA",
"FLEBITIS Y TROMBOFLEBITIS DE OTROS SITIOS",
"FLEBITIS Y TROMBOFLEBITIS DE OTROS VASOS PROFUNDOS DE LOS MIEMBROS INFERIORES",
"FRACTURA DE COSTILLA",
"FRACTURA DE HUESO DEL METATARSO",
"FRACTURA DE LA BÃ“VEDA DEL CRÃNEO",
"FRACTURA DE LA CLAVÃCULA",
"FRACTURA DE LA CLAVICULA DEBIDA A TRAUMATISMO DEL NACIMIENTO",
"FRACTURA DE LA DIÃFISIS DE LA TIBIA",
"FRACTURA DE LA DIÃFISIS DEL CÃšBITO Y DEL RADIO",
"FRACTURA DE LA DIÃFISIS DEL CÃšBITO",
"FRACTURA DE LA DIÃFISIS DEL FÃ‰MUR",
"FRACTURA DE LA DIÃFISIS DEL HÃšMERO",
"FRACTURA DE LA DIÃFISIS DEL RADIO",
"FRACTURA DE LA EPÃFISIS INFERIOR DE LA TIBIA",
"FRACTURA DE LA EPÃFISIS INFERIOR DEL CÃšBITO Y DEL RADIO",
"FRACTURA DE LA EPÃFISIS INFERIOR DEL FÃ‰MUR",
"FRACTURA DE LA EPÃFISIS INFERIOR DEL HÃšMERO",
"FRACTURA DE LA EPÃFISIS INFERIOR DEL RADIO",
"FRACTURA DE LA EPÃFISIS SUPERIOR DEL CÃšBITO",
"FRACTURA DE LA EPÃFISIS SUPERIOR DEL HÃšMERO",
"FRACTURA DE LA EPÃFISIS SUPERIOR DEL RADIO",
"FRACTURA DE LA PIERNA, PARTE NO ESPECIFICADA",
"FRACTURA DE LA PRIMERA VÃ‰RTEBRA CERVICAL",
"FRACTURA DE LA RÃ“TULA",
"FRACTURA DE LOS HUESOS DE LA NARIZ",
"FRACTURA DE LOS HUESOS DE OTRO(S) DEDO(S) DEL PIE",
"FRACTURA DE LOS HUESOS DEL DEDO GORDO DEL PIE",
"FRACTURA DE MIEMBRO INFERIOR, NIVEL NO ESPECIFICADO",
"FRACTURA DE MIEMBRO SUPERIOR, NIVEL NO ESPECIFICADO",
"FRACTURA DE OTRAS PARTES DE LA PIERNA",
"FRACTURA DE OTRAS PARTES DEL HOMBRO Y DEL BRAZO",
"FRACTURA DE OTRAS PARTES DEL TÃ“RAX Ã“SEO",
"FRACTURA DE OTRAS PARTES Y DE LAS NO ESPECIFICADAS DE LA COLUMNA LUMBAR Y DE LA PELVIS",
"FRACTURA DE OTRAS VÃ‰RTEBRAS CERVICALES ESPECIFICADAS",
"FRACTURA DE OTRO DEDO DE LA MANO",
"FRACTURA DE OTROS HUESOS DEL CRÃNEO Y DE LA CARA",
"FRACTURA DE OTROS HUESOS METACARPIANOS",
"FRACTURA DE REGION NO ESPECIFICADA DEL CUERPO",
"FRACTURA DE VÃ‰RTEBRA LUMBAR",
"FRACTURA DE VÃ‰RTEBRA TORÃCICA",
"FRACTURA DEL ACETÃBULO",
"FRACTURA DEL CALCANEO",
"FRACTURA DEL CRÃNEO Y DE LOS HUESOS DE LA CARA, PARTE NO ESPECIFICADA",
"FRACTURA DEL CUELLO DEL FÃ‰MUR",
"FRACTURA DEL FÃ‰MUR, PARTE NO ESPECIFICADA",
"FRACTURA DEL HUESO ILIACO",
"FRACTURA DEL MALEOLO EXTERNO",
"FRACTURA DEL MALEOLO INTERNO",
"FRACTURA DEL MAXILAR INFERIOR",
"FRACTURA DEL PIE, NO ESPECIFICADA",
"FRACTURA DEL PUBIS",
"FRACTURA PERTROCANTERIANA",
"FRACTURA SUBTROCANTERIANA",
"FRACTURAS MÃšLTIPLES DE COLUMNA CERVICAL",
"FRACTURAS MÃšLTIPLES DE COSTILLAS",
"FRACTURAS MÃšLTIPLES DE LA COLUMNA LUMBAR Y DE LA PELVIS",
"FRACTURAS QUE AFECTAN MÃšLTIPLES REGIONES DE AMBOS MIEMBROS SUPERIORES",
"FRACTURAS QUE AFECTAN MÃšLTIPLES REGIONES DE UN MIEMBRO SUPERIOR",
"GANGRENA GASEOSA",
"GANGRENA, NO CLASIFICADA EN OTRA PARTE",
"GASTRITIS AGUDA HEMORRAGICA",
"GASTRITIS ALCOHOLICA",
"GASTRITIS CRONICA, NO ESPECIFICADA",
"GASTRITIS, NO ESPECIFICADA",
"GASTROENTERITIS Y COLITIS DE ORIGEN NO ESPECIFICADO",
"GASTROSTOMÃA",
"GLAUCOMA, NO ESPECIFICADO",
"GONARTROSIS, NO ESPECIFICADA",
"GRANULOMA ANULAR",
"HALLAZGOS ANORMALES EN EL EXAMEN CITOLOGICO E HISTOLOGICO DE LA ORINA",
"HALLAZGOS ANORMALES EN EL EXAMEN MICROBIOLOGICO DE LA ORINA",
"HEMATEMESIS",
"HEMATOMA DE HERIDA QUIRURGICA OBSTETRICA",
"HEMATURIA, NO ESPECIFICADA",
"HEMIPLEJIA, NO ESPECIFICADA",
"HEMOPERITONEO",
"HEMORRAGIA CONJUNTIVAL",
"HEMORRAGIA DEL ANO Y DEL RECTO",
"HEMORRAGIA DEL VITREO",
"HEMORRAGIA GASTROINTESTINAL, NO ESPECIFICADA",
"HEMORRAGIA INTRACEREBRAL EN HEMISFERIO, CORTICAL",
"HEMORRAGIA INTRACEREBRAL EN HEMISFERIO, NO ESPECIFICADA",
"HEMORRAGIA INTRACEREBRAL EN HEMISFERIO, SUBCORTICAL",
"HEMORRAGIA INTRACRANEAL (NO TRAUMATICA), NO ESPECIFICADA",
"HEMORRAGIA INTRAENCEFALICA DE LOCALIZACIONES MULTIPLES",
"HEMORRAGIA INTRAENCEFALICA EN CEREBELO",
"HEMORRAGIA SUBARACNOIDEA DE ARTERIA COMUNICANTE POSTERIOR",
"HEMORRAGIA SUBARACNOIDEA TRAUMÃTICA",
"HEMORRAGIA SUBARACNOIDEA, NO ESPECIFICADA",
"HEMORRAGIA SUBDURAL (AGUDA) (NO TRAUMATICA)",
"HEMORRAGIA SUBDURAL TRAUMÃTICA",
"HEMORRAGIA Y HEMATOMA QUE COMPLICAN UN PROCEDIMIENTO, NO CLASIFICADOS EN OTRA PARTE",
"HEMORRAGIA, NO CLASIFICADA EN OTRA PARTE",
"HEMORROIDES DE CUARTO GRADO",
"HEMORROIDES DE SEGUNDO GRADO",
"HEMORROIDES DE TERCER GRADO",
"HEMORROIDES, SIN OTRA ESPECIFICACIÃ“N",
"HEPATITIS VIRAL AGUDA, NO ESPECIFICADA",
"HERIDA DE DEDO(S) DE LA MANO, CON DAÃ‘O DE LA(S) UÃ‘A(S)",
"HERIDA DE DEDO(S) DE LA MANO, SIN DAÃ‘O DE LA(S) UÃ‘A(S)",
"HERIDA DE DEDO(S) DEL PIE CON DAÃ‘O DE LA(S) UÃ‘A(S)",
"HERIDA DE DEDO(S) DEL PIE SIN DAÃ‘O DE LA(S) UÃ‘A(S)",
"HERIDA DE LA CABEZA, PARTE NO ESPECIFICADA",
"HERIDA DE LA MAMA",
"HERIDA DE LA MUÃ‘ECA Y DE LA MANO, PARTE NO ESPECIFICADA",
"HERIDA DE LA PARED ABDOMINAL",
"HERIDA DE LA PIERNA, PARTE NO ESPECIFICADA",
"HERIDA DE MIEMBRO INFERIOR, NIVEL NO ESPECIFICADO",
"HERIDA DE MIEMBRO SUPERIOR, NIVEL NO ESPECIFICADO",
"HERIDA DE OTRAS PARTES DE LA CABEZA",
"HERIDA DE OTRAS PARTES DE LA PIERNA",
"HERIDA DE OTRAS PARTES DEL ANTEBRAZO",
"HERIDA DE OTRAS PARTES DEL PIE",
"HERIDA DE OTRAS PARTES DEL TORAX",
"HERIDA DE REGION NO ESPECIFICADA DEL CUERPO",
"HERIDA DEL BRAZO",
"HERIDA DEL CODO",
"HERIDA DEL CUERO CABELLUDO",
"HERIDAS MULTIPLES DEL ANTEBRAZO",
"HERNIA ABDOMINAL NO ESPECIFICADA, CON OBSTRUCCION, SIN GANGRENA",
"HERNIA ABDOMINAL NO ESPECIFICADA, SIN OBSTRUCCION NI GANGRENA",
"HERNIA INCISIONAL SIN OBSTRUCCIÃ“N O GANGRENA",
"HERNIA INGUINAL BILATERAL, SIN OBSTRUCCION NI GANGRENA",
"HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA, CON OBSTRUCCION, SIN GANGRENA",
"HERNIA INGUINAL UNILATERAL O NO ESPECIFICADA, SIN OBSTRUCCION NI GANGRENA",
"HERNIA UMBILICAL CON OBSTRUCCION, SIN GANGRENA",
"HERNIA VENTRAL CON OBSTRUCCION, SIN GANGRENA",
"HERPES ZOSTER CON OTRAS COMPLICACIONES",
"HERPES ZOSTER SIN COMPLICACIONES",
"HIDROCEFALO DE PRESION NORMAL",
"HIDRONEFROSIS CON ESTRECHEZ URETERAL, NO CLASIFICADA EN OTRA PARTE",
"HIPERGLICEMIA, NO ESPECIFICADA",
"HIPERGLICERIDEMIA PURA",
"HIPERLIPIDEMIA MIXTA",
"HIPEROSMOLARIDAD E HIPERNATREMIA",
"HIPERPLASIA DE LA PROSTATA",
"HIPERPOTASEMIA",
"HIPERTENSION ESENCIAL (PRIMARIA)",
"HIPERTENSION INTRACRANEAL BENIGNA",
"HIPERTENSION PULMÃ“NAR PRIMARIA",
"HIPOGLICEMIA SIN COMA, INDUCIDA POR DROGAS",
"HIPOGLICEMIA, NO ESPECIFICADA",
"HIPOPARATIROIDISMO CONSECUTIVO A PROCEDIMIENTOS",
"HIPOPOTASEMIA",
"HIPOSMOLARIDAD E HIPONATREMIA",
"HIPOTENSION DEBIDA A DROGAS",
"HIPOTENSION IDIOPATICA",
"HIPOTENSION ORTOSTATICA",
"HIPOTENSION, NO ESPECIFICADA",
"HIPOTERMIA",
"HIPOTIROIDISMO, NO ESPECIFICADO",
"HISTORIA PERSONAL DE TUMOR MALIGNO DE Ã“RGANOS DIGESTIVOS",
"ICTERICIA NO ESPECIFICADA",
"ILEO PARALITICO",
"ILEO, NO ESPECIFICADO",
"INCIDENTE (CORTE, PUNCION, PERFORACION O HEMORRAGIA) DURANTE DIALISIS RENAL U OTRA PERFUSION",
"INCONTINENCIA URINARIA, NO ESPECIFICADA",
"INFARTO AGUDO DEL MIOCARDIO, SIN OTRA ESPECIFICACION",
"INFARTO AGUDO TRANSMURAL DEL MIOCARDIO DE OTROS SITIOS",
"INFARTO ANTIGUO DEL MIOCARDIO",
"INFARTO CEREBRAL, NO ESPECIFICADO",
"INFARTO SUBENDOCARDICO AGUDO DEL MIOCARDIO",
"INFARTO SUBSECUENTE DEL MIOCARDIO DE LA PARED ANTERIOR",
"INFARTO TRANSMURAL AGUDO DEL MIOCARDIO DE LA PARED INFERIOR",
"INFECCIÃ“N BACTERIANA, NO ESPECIFICADA",
"INFECCIÃ“N DEBIDA AL VIRUS DEL HERPES, NO ESPECIFICADA",
"INFECCIÃ“N VIRAL, NO ESPECIFICADA",
"INFECCION AGUDA DE LAS VÃAS RESPIRATORIAS SUPERIORES, NO ESPECIFICADA",
"INFECCION AGUDA NO ESPECIFICADA DE LAS VÃAS RESPIRATORIAS INFERIORES",
"INFECCION DE OTRAS PARTES DE LAS VÃAS URINARIAS EN EL EMBARAZO",
"INFECCION DE VÃAS URINARIAS, SITIO NO ESPECIFICADO",
"INFECCION DEL MUÃ‘ON DE AMPUTACION",
"INFECCION LOCAL DE LA PIEL Y DEL TEJIDO SUBCUTÃNEO, NO ESPECIFICADA",
"INFECCION NO ESPECIFICADA DE LAS VÃAS URINARIAS EN EL EMBARAZO",
"INFECCION POSTRAUMATICA DE HERIDA, NO CLASIFICADA EN OTRA PARTE",
"INFECCION Y REACCION INFLAMATORIA DEBIDAS A PROTESIS ARTICULAR INTERNA",
"INFLUENZA CON OTRAS MANIFESTACIONES, VIRUS NO IDENTIFICADO",
"INQUIETUD Y AGITACION",
"INSOMNIO NO ORGANICO",
"INSUFICIENCIA CARDIACA CONGESTIVA",
"INSUFICIENCIA CARDIACA, NO ESPECIFICADA",
"INSUFICIENCIA HEPATICA ALCOHOLICA",
"INSUFICIENCIA HEPATICA CRONICA",
"INSUFICIENCIA PULMÃ“NAR CRONICA CONSECUTIVA A CIRUGÃA",
"INSUFICIENCIA RENAL AGUDA, NO ESPECIFICADA",
"INSUFICIENCIA RENAL NO ESPECIFICADA",
"INSUFICIENCIA RESPIRATORIA AGUDA",
"INSUFICIENCIA RESPIRATORIA, NO ESPECIFICADA",
"INSUFICIENCIA VENOSA (CRONICA) (PERIFERICA)",
"INTOXICACION ALCOHOLICA LEVE",
"INTOXICACION ALCOHOLICA MODERADA",
"INTOXICACION ALCOHOLICA SEVERA",
"ISQUEMIA CEREBRAL TRANSITORIA, SIN OTRA ESPECIFICACION",
"ISQUEMIA MIOCARDICA TRANSITORIA DEL RECIEN NACIDO",
"ISQUEMIA SILENTE DEL MIOCARDIO",
"LEIOMIOMA DEL ÃšTERO, SIN OTRA ESPECIFICACIÃ“N",
"LESIÃ“N (NEOPLASIA) DE SITIOS CONTIGUOS DE LA LARINGE",
"LESIÃ“N (NEOPLASIA) DE SITIOS CONTIGUOS DE LAS VÃAS BILIARES",
"LESIÃ“N (NEOPLASIA) DE SITIOS CONTIGUOS DE LOS BRONQUIOS Y DEL PULMÃ“N",
"LESIÃ“N (NEOPLASIA) DE SITIOS CONTIGUOS DEL ANO, DEL CONDUCTO ANAL Y DEL RECTO",
"LESIÃ“N (NEOPLASICA) DE SITIOS CONTIGUOS DE LOS Ã“RGANOS GENITALES FEMENINOS",
"LEUCEMIA CRÃ“NICA, CÃ‰LULAS DE TIPO NO ESPECIFICADO",
"LINFADENITIS AGUDA DE OTROS SITIOS",
"LINFADENITIS AGUDA DEL MIEMBRO SUPERIOR",
"LINFEDEMA, NO CLASIFICADO EN OTRA PARTE",
"LIPOMATOSIS, NO CLASIFICADA EN OTRA PARTE",
"LUMBAGO CON CIÃTICA",
"LUMBAGO NO ESPECIFICADO",
"LUPUS ERITEMATOSO SISTEMICO CON COMPROMISO DE Ã“RGANOS O SISTEMAS",
"LUXAC., ESGUINCE O TORCED. ARTICULAC. Y LIGAM. NO ESPECIFIC. MIEMBRO INFERIOR, NIVEL NO ESPECIFIC.",
"LUXACION CONGENITA DE LA CADERA, BILATERAL",
"LUXACION CONGENITA DE LA CADERA, NO ESPECIFICADA",
"LUXACION DE LA ARTICULACION ACROMIOCLAVICULAR",
"LUXACION DE LA ARTICULACION DEL HOMBRO",
"LUXACION DE LA CADERA",
"LUXACION DE LA ROTULA",
"LUXACION DE OTRAS PARTES Y DE LAS NO ESPECIFICADAS DE LA CABEZA",
"LUXACION DEL CODO, NO ESPECIFICADA",
"LUXACIONES MULTIPLES DE DEDOS DE LA MANO",
"LUXACIONES, TORCEDURAS Y ESGUINCES QUE AFECTAN LA CABEZA CON EL CUELLO",
"MALESTAR Y FATIGA",
"MARCHA ATAXICA",
"MAREO Y DESVANECIMIENTO",
"MASA NO ESPECIFICADA EN LA MAMA",
"MASTODINIA",
"MEDICAMENTO O SUSTANCIA BIOLOGICA CONTAMINADO, ADMINISTRADO POR MEDIOS NO ESPECIFICADOS",
"MELENA",
"MIALGIA",
"MICOSIS , NO ESPECIFICADA",
"MICOSIS SUPERFICIAL, SIN OTRA ESPECIFICACIÃ“N",
"MIELOMA MÃšLTIPLE",
"MIGRAÃ‘A CON AURA (MIGRAÃ‘A CLASICA)",
"MIGRAÃ‘A, NO ESPECIFICADA",
"MIOCLONIA",
"MONONEUROPATIA DEL MIEMBRO INFERIOR, SIN OTRA ESPECIFICACION",
"MORDEDURA DE RATA, OCURRIDA EN UNA VIVIENDA",
"MORDEDURA O ATAQUE DE OTROS MAMIFEROS, OCURRIDO EN UNA INSTITUCION RESIDENCIAL",
"MORDEDURA O ATAQUE DE PERRO, OCURRIDO EN OTRO LUGAR ESPECIFICADO",
"MORDEDURA O ATAQUE DE PERRO, OCURRIDO EN UN COMERCIO O EN UN AREA DE SERVICIOS",
"MORDEDURA O PICADURA DE INSECTOS Y DE OTROS ARTROPODOS NO VENENOSOS, OCURRIDA EN OTRO LUGAR ESPECIFICADO",
"MORDEDURA O PICADURA DE INSECTOS Y DE OTROS ARTROPODOS NO VENENOSOS, OCURRIDA EN UNA ESCUELA, EN OTRA INSTITUCION, O EN UN AREA ADMINISTRATIVA PUBLICA",
"MORDEDURA O PICADURA DE INSECTOS Y DE OTROS ARTROPODOS NO VENENOSOS, OCURRIDA EN UNA VIVIENDA",
"NAUSEA Y VOMITO",
"NECROSIS ARTERIAL",
"NECROSIS DEL MUÃ‘ON DE AMPUTACION",
"NEFRITIS TUBULOINTERSTICIAL AGUDA",
"NEUMONIA BACTERIANA, NO ESPECIFICADA",
"NEUMONIA EN OTRAS ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"NEUMONIA, NO ESPECIFICADA",
"NEUMOTORAX TRAUMATICO",
"NEURALGIA DEL TRIGEMINO",
"NEURALGIA Y NEURITIS NO ESPECIFICADAS",
"NEURONITIS VESTIBULAR",
"NEUROPATIA AUTONOMA PERIFERICA IDIOPATICA",
"NEUROPATIA AUTONOMICA EN ENFERMEDADES METABOLICAS Y ENDOCRINAS",
"NEUROPATIA INTERCOSTAL",
"NEVO MELANOCITICO DEL TRONCO",
"OBESIDAD DEBIDA A EXCESO DE CALORÃAS",
"OBSERVACION POR SOSPECHA DE EFECTOS TOXICOS DE SUSTANCIAS INGERIDAS",
"OBSERVACION POR SOSPECHA DE OTRAS ENFERMEDADES Y AFECCIONES",
"OBSERVACION POR SOSPECHA DE TUMOR MALIGNO",
"OBSTRUCCION INTESTINAL POSTOPERATORIA",
"OCLUSION Y ESTENOSIS DE ARTERIA CAROTIDA",
"ORQUITIS, EPIDIDIMITIS Y ORQUIEPIDIDIMITIS SIN ABSCESO",
"OSTEOCONDROPATIA, NO ESPECIFICADA",
"OSTEOCONDROSIS DE LA COLUMNA VERTEBRAL DEL ADULTO",
"OSTEOCONDROSIS JUVENIL DEL HUMERO",
"OSTEOMIELITIS SUBAGUDA",
"OSTEOMIELITIS, NO ESPECIFICADA",
"OTITIS EXTERNA AGUDA, NO INFECCIOSA",
"OTITIS EXTERNA MALIGNA",
"OTITIS EXTERNA, SIN OTRA ESPECIFICACION",
"OTITIS MEDIA EN ENFERMEDADES VIRALES CLASIFICADAS EN OTRA PARTE",
"OTITIS MEDIA SUPURATIVA AGUDA",
"OTITIS MEDIA, NO ESPECIFICADA",
"OTRA DIABETES MELLITUS ESPECIFICADA, CON CETOACIDOSIS",
"OTRA DIABETES MELLITUS ESPECIFICADA, CON COMPLICACIONES CIRCULATORIAS PERIFÃ‰RICAS",
"OTRA DIABETES MELLITUS ESPECIFICADA, SIN MENCIÃ“N DE COMPLICACIÃ“N",
"OTRA REACCION ADVERSA A ALIMENTOS, NO CLASIFICADA EN OTRA PARTE",
"OTRAS (TENO)SINOVITIS INFECCIOSAS",
"OTRAS ALTERACIONES DEL HABLA Y LAS NO ESPECIFICADAS",
"OTRAS ANEMIAS ESPECIFICADAS",
"OTRAS ANORMALIDADES FECALES",
"OTRAS ARRITMIAS CARDIACAS ESPECIFICADAS",
"OTRAS CARDIOMIOPATIAS HIPERTROFICAS",
"OTRAS CIRROSIS DEL HÃGADO Y LAS NO ESPECIFICADAS",
"OTRAS COLECISTITIS",
"OTRAS COLELITIASIS",
"OTRAS COLITIS Y GASTROENTERITIS NO INFECCIOSAS ESPECIFICADAS",
"OTRAS COMPLICACIONES DE DISPOSITIVOS PROTESICOS, IMPLANTES E INJERTOS ORTOPÃ‰DICOS INTERNOS",
"OTRAS COMPLICACIONES ESPECIFICADAS DE LA ATENCIÃ“N MEDICA Y QUIRURGICA, NCOP",
"OTRAS CONSULTAS ESPECIFICADAS",
"OTRAS CONVULSIONES Y LAS NO ESPECIFICADAS",
"OTRAS DEGENERACIONES DE DISCO CERVICAL",
"OTRAS DISFUNCIONES TESTICULARES",
"OTRAS ENFERMEDADES BACTERIANAS ESPECIFICADAS",
"OTRAS ENFERMEDADES CEREBROVASCULARES ESPECIFICADAS",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL ANO Y DEL RECTO",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL ESOFAGO",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL HÃGADO",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL INTESTINO",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL PANCREAS",
"OTRAS ENFERMEDADES ESPECIFICADAS DEL SISTEMA DIGESTIVO",
"OTRAS ENFERMEDADES PULMÃ“NARES INTERSTICIALES CON FIBROSIS",
"OTRAS ENFERMEDADES PULMÃ“NARES OBSTRUCTIVAS CRÃ“NICAS ESPECIFICADAS",
"OTRAS ENFERMEDADES VASCULARES PERIFERICAS ESPECIFICADAS",
"OTRAS ENFERMEDADES VIRALES ESPECIFICADAS",
"OTRAS ESPONDILOPATÃAS ESPECIFICADAS",
"OTRAS FIEBRES ESPECIFICADAS",
"OTRAS FORMAS ESPECIFICADAS DE ANGINA DE PECHO",
"OTRAS FUSIONES DE LA COLUMNA VERTEBRAL",
"OTRAS GASTRITIS AGUDAS",
"OTRAS GASTRITIS",
"OTRAS GASTROENTERITIS Y COLITIS NO ESPECIFICADAS DE ORIGEN INFECCIOSO",
"OTRAS HIDRONEFROSIS Y LAS NO ESPECIFICADAS",
"OTRAS HIPOGLICEMIAS",
"OTRAS INFECCIONES BACTERIANAS DE SITIO NO ESPECIFICADO",
"OTRAS INFECCIONES LOCALES ESPECIFICADAS DE LA PIEL Y DEL TEJIDO SUBCUTÃNEO",
"OTRAS INFECCIONES Y LAS NO ESPECIFICADAS DE LAS VÃAS GENITOURINARIAS EN EL EMBARAZO",
"OTRAS ISQUEMIAS CEREBRALES TRANSITORIAS Y SINDROMES AFINES",
"OTRAS LESIÃ“NES DEL NERVIO MEDIANO",
"OTRAS OBSTRUCCIONES DEL INTESTINO",
"OTRAS OBSTRUCCIONES INTESTINALES Y LAS NO ESPECIFICADAS",
"OTRAS OSTEOMIELITIS AGUDAS",
"OTRAS OSTEOMIELITIS CRÃ“NICAS",
"OTRAS OTITIS MEDIAS CRÃ“NICAS SUPURATIVAS",
"OTRAS PANCREATITIS AGUDAS",
"OTRAS QUERATITIS SUPERFICIALES SIN CONJUNTIVITIS",
"OTRAS REACCIONES AL ESTRES GRAVE",
"OTRAS SINUSITIS AGUDAS",
"OTRAS TROMBOCITOPENIAS PRIMARIAS",
"OTRAS TUBERCULOSIS RESPIRATORIAS, CONFIRMADAS BACTERIOL+OGICA E HISTOLÃ“GICAMENTE",
"OTRO DOLOR CRONICO",
"OTROS DEFECTOS ESPECIFICADOS DE LA COAGULACIÃ“N",
"OTROS DESGARROS (NO TRAUMÃ“TICOS) DE MÃšSCULO",
"OTROS DESPRENDIMIENTOS DE LA RETINA",
"OTROS DOLORES ABDOMINALES Y LOS NO ESPECIFICADOS",
"OTROS DOLORES EN EL PECHO",
"OTROS ESTADOS EPILEPTICOS",
"OTROS ESTADOS POSTQUIRURGICOS ESPECIFICADOS",
"OTROS EXAMENES PARA FINES ADMINISTRATIVOS",
"OTROS HALLAZGOS ANORMALES EN LA ORINA Y LOS NO ESPECIFICADOS",
"OTROS HALLAZGOS ANORMALES ESPECIFICADOS EN LA QUIMICA SANGUINEA",
"OTROS PENFIGOIDES",
"OTROS QUISTES FOLICULARES DE LA PIEL Y DEL TEJIDO SUBCUTÃNEO",
"OTROS SINDROMES DE CEFALEA ESPECIFICADOS",
"OTROS SINT. Y SIGNOS QUE INVOLUCRAN LA FUNCION COGNOSC.Y LA CONCIENCIA Y LOS NO ESPECIFIC.",
"OTROS TIPOS DE ENFERMEDAD DE CROHN",
"OTROS TIPOS DE OBESIDAD",
"OTROS TRAST DE LA SANGRE Y DE ORGANOS HEMATOPOYETICOS EN ENF CLASIFICADAS EN OTROS CAPITULOS",
"OTROS TRASTORNOS AFECTIVOS BIPOLARES",
"OTROS TRASTORNOS DE ANSIEDAD ESPECIFICADOS",
"OTROS TRASTORNOS DE ANSIEDAD MIXTOS",
"OTROS TRASTORNOS DE DISCO CERVICAL",
"OTROS TRASTORNOS DE LA FUNCION VESTIBULAR",
"OTROS TRASTORNOS DE LA INGESTION DE ALIMENTOS",
"OTROS TRASTORNOS DE LA ORBITA",
"OTROS TRASTORNOS DE LAS RAICES Y PLEXOS NERVIOSOS",
"OTROS TRASTORNOS DE PIEL Y DEL TEJIDO SUBCUTANEO EN ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"OTROS TRASTORNOS DEL EQUILIBRIO DE LOS ELECTROLITOS Y DE LOS LÃQUIDOS, NCOP",
"OTROS TRASTORNOS DEL HUMOR (AFECTIVOS), AISLADOS",
"OTROS TRASTORNOS DEL HUMOR (AFECTIVOS), ESPECIFICADOS",
"OTROS TRASTORNOS DEL SISTEMA GENITOURINARIO CONSECUTIVOS A PROCEDIMIENTOS",
"OTROS TRASTORNOS DEL SISTEMA NERVIOSO, NO CLASIFICADOS EN OTRA PARTE",
"OTROS TRASTORNOS DISOCIATIVOS (DE CONVERSION)",
"OTROS TRASTORNOS ENDOCRINOS ESPECIFICADOS",
"OTROS TRASTORNOS ESPECIF DEL APARATO CIRCULATORIO EN ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"OTROS TRASTORNOS ESPECIFICADOS DE LA MAMA",
"OTROS TRASTORNOS ESPECIFICADOS DE LA NARIZ Y DE LOS SENOS PARANASALES",
"OTROS TRASTORNOS ESPECIFICADOS DE LA PROSTATA",
"OTROS TRASTORNOS ESPECIFICADOS DE LOS LEUCOCITOS",
"OTROS TRASTORNOS ESPECIFICADOS DE LOS MÃšSCULOS",
"OTROS TRASTORNOS ESPECIFICADOS DEL ENCEFALO",
"OTROS TRASTORNOS ESPECIFICADOS DEL METABOLISMO DE LOS CARBOHIDRATOS",
"OTROS TRASTORNOS ESPECIFICADOS DEL OIDO EN ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"OTROS TRASTORNOS ESPECIFICADOS DEL OJO Y SUS ANEXOS",
"OTROS TRASTORNOS ESPECIFICADOS DEL SISTEMA NERVIOSO CENTRAL",
"OTROS TRASTORNOS ESPECIFICADOS DEL SISTEMA URINARIO",
"OTROS TRASTORNOS FOBICOS DE ANSIEDAD",
"OTROS TRASTORNOS INTERNOS DE LA RODILLA",
"OTROS TRASTORNOS RI?ON Y URETER EN ENF INFECCIOSAS Y PARASITARIAS CLASIFICADAS EN OTROS CAPITULOS",
"OTROS TRASTORNOS VASCULARES DEL INTESTINO",
"OTROS TRAUMATISMOS DE LA CABEZA, ESPECIFICADOS",
"OTROS TRAUMATISMOS DE REGION NO ESPECIFICADA DEL CUERPO",
"OTROS TRAUMATISMOS DEL OJO Y DE LA ORBITA",
"OTROS TRAUMATISMOS ESPECIFICADOS QUE AFECTAN MULTIPLES REGIONES DEL CUERPO",
"OTROS TRAUMATISMOS SUPERFICIALES DE LA PARED ANTERIOR DEL TORAX",
"OTROS TRAUMATISMOS SUPERFICIALES DE LA PARED POSTERIOR DEL TORAX",
"OTROS TRAUMATISMOS SUPERFICIALES DEL ABDOMEN, DE LA REGION LUMBOSACRA Y DE LA PELVIS",
"OTROS TRAUMATISMOS SUPERFICIALES DEL HOMBRO Y DEL BRAZO",
"OTROS TUM.ESPECIF.COMPORT.INCIERTO/DESCONOC.TEJ. LINFAT, ORG. HEMATOPOY. Y TEJIDOS AFINES",
"OTROS VERTIGOS PERIFERICOS",
"PALPITACIONES",
"PANCREATITIS AGUDA INDUCIDA POR EL ALCOHOL",
"PANCREATITIS AGUDA NO ESPECIFICADA",
"PANCREATITIS BILIAR AGUDA",
"PARALISIS DE BELL",
"PARALISIS DEL NERVIO MOTOR OCULAR EXTERNO (VI PAR)",
"PARESTESIA DE LA PIEL",
"PARO CARDIACO, NO ESPECIFICADO",
"PAROTIDITIS, SIN COMPLICACIONES",
"PIELONEFRITIS CRONICA NO OBSTRUCTIVA ASOCIADA CON REFLUJO",
"PIODERMA",
"POLIARTRITIS, NO ESPECIFICADA",
"POLIARTROSIS, NO ESPECIFICADA",
"POLICONDRITIS RECIDIVANTE",
"POLIMIOSITIS",
"POLINEUROPATIA, NO ESPECIFICADA",
"PREPUCIO REDUNDANTE, FIMOSIS Y PARAFIMOSIS",
"PRESENCIA DE ALCOHOL EN LA SANGRE, NIVEL NO ESPECIFICADO",
"PRESENCIA DE ANGIOPLASTIA, INJERTOS Y PROTESIS CORONARIAS",
"PRESENCIA DE MARCAPASO CARDÃACO",
"PROBLEMAS RELACIONADOS CON EL ESTRES, NO CLASIFICADOS EN OTRA PARTE",
"PROSTATITIS AGUDA",
"PROSTATOCISTITIS",
"PRUEBA DE ALCOHOL O DROGAS EN LA SANGRE",
"PRUEBA Y AJUSTE DE DISPOSITIVO PROTESICO EXTERNO NO ESPECIFICADO",
"PSEUDOCOXALGIA",
"QUEMADURA DE LA MUÃ‘ECA Y DE LA MANO, DE PRIMER GRADO",
"QUEMADURA DE LAS VÃAS RESPIRATORIAS, PARTE NO ESPECIFICADA",
"QUEMADURA DE REGION DEL CUERPO Y GRADO NO ESPECIFICADOS",
"QUEMADURA DE SEGUNDO GRADO, REGION DEL CUERPO NO ESPECIFICADA",
"QUEMADURA DEL HOMBRO Y MIEMBRO SUPERIOR, DE PRIMER GRADO, EXCEPTO DE LA MUÃ‘ECA Y DE LA MANO",
"QUEMADURAS QUE AFECTAN MENOS DEL 10% DE LA SUPERFICIE DEL CUERPO",
"QUISTE DEL COLEDOCO",
"QUISTE EPIDERMICO",
"QUISTE FOLICULAR DE LA PIEL Y DEL TEJIDO SUBCUTÃNEO, SIN OTRA ESPECIFICACION",
"QUISTE SINOVIAL DEL HUECO POPLÃTEO (DE BAKER)",
"RADIODERMATITIS AGUDA",
"REACCION AL ESTRES AGUDO",
"RETENCION DE ORINA",
"RETINOPATIAS DEL FONDO Y CAMBIOS VASCULARES RETINIANOS",
"RINOFARINGITIS AGUDA (RESFRIADO COMUN)",
"RUPTURA ESPONTÃNEA DE TENDONES EXTENSORES",
"RUPTURA TRAUMATICA DE DISCO INTERVERTEBRAL LUMBAR",
"SECUELAS DE ENFERMEDAD CEREBROVASCULAR, NO ESPECIFICADA COMO HEMORRAGICA U OCLUSIVA",
"SECUELAS DE FRACTURA DEL FEMUR",
"SECUELAS DE HERIDA DE MIEMBRO INFERIOR",
"SECUELAS DE INFARTO CEREBRAL",
"SECUELAS DE LESIONES AUTO INFLIGIDAS",
"SECUELAS DE OTRAS ENFERMEDADES CEREBROVASCULARES, ESPECIFICADAS",
"SECUELAS DE TRAUMATISMO DE TENDON Y MUSCULO DE MIEMBRO SUPERIOR",
"SECUELAS DE TRAUMATISMO NO ESPECIFICADO DE LA CABEZA",
"SECUELAS DE TRAUMATISMO SUPERFICIAL DE LA CABEZA",
"SENILIDAD",
"SEPTICEMIA, NO ESPECIFICADA",
"SINCOPE Y COLAPSO",
"SINDROME DE CEFALEA EN RACIMOS",
"SINDROME DE LA ARTICULACIÃ“N CONDROCOSTAL (TIETZE)",
"SINDROME DEL COLON IRRITABLE CON DIARREA",
"SINDROME DEL COLON IRRITABLE SIN DIARREA",
"SINDROME DEL MANGUITO ROTATORIO",
"SINDROME DEL TUNEL CALCANEO",
"SINDROME DEL TUNEL CARPIANO",
"SINDROME HEPATORRENAL",
"SINDROMES VERTIGINOSOS EN ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"SINOVITIS Y TENOSINOVITIS, NO ESPECIFICADA",
"SINUSITIS AGUDA, NO ESPECIFICADA",
"SOMNOLENCIA",
"TAQUICARDIA PAROXISTICA, NO ESPECIFICADA",
"TAQUICARDIA SUPRAVENTRICULAR",
"TAQUICARDIA VENTRICULAR",
"TAQUICARDIA, NO ESPECIFICADA",
"TEMBLOR NO ESPECIFICADO",
"TENDINITIS CALCIFICANTE DEL HOMBRO",
"TENDINITIS DEL BICEPS",
"TENDINITIS DEL GLUTEO",
"TENDINITIS PERONEAL",
"TENDINITIS ROTULIANA",
"TENESMO VESICAL",
"TENOSINOVITIS DE ESTILOIDES RADIAL (DE QUERVAIN)",
"TORSION DEL TESTICULO",
"TORTICOLIS ESPASMODICA",
"TRANSFUSIÃ“N DE SANGRE, (SIN DIAGNÃ“STICO INFORMADO)",
"TRAQUEITIS AGUDA",
"TRAQUEOSTOMIA",
"TRAST.MENT.Y DE COMPORT. DEBIDOS A INTOXIC. AGUDA POR USO DE OTROS ESTIMULANTES, INCL. CAFEINA",
"TRASTORNO AFECTIVO BIPOLAR, ACTUALMENTE EN REMISION",
"TRASTORNO AFECTIVO BIPOLAR, EPISODIO DEPRESIVO GRAVE PRESENTE SIN SINTOMAS PSICÃ“TICOS",
"TRASTORNO DE ANSIEDAD GENERALIZADA",
"TRASTORNO DE ANSIEDAD, NO ESPECIFICADO",
"TRASTORNO DE LA CONDUCTA, NO ESPECIFICADO",
"TRASTORNO DE LA FUNCION VESTIBULAR, NO ESPECIFICADO",
"TRASTORNO DE LA PERSONALIDAD ANSIOSA (EVASIVA, ELUSIVA)",
"TRASTORNO DE LA PERSONALIDAD EMOCIONALMENTE INESTABLE",
"TRASTORNO DE LA PROSTATA, NO ESPECIFICADO",
"TRASTORNO DE LA ROTULA, NO ESPECIFICADO",
"TRASTORNO DE LA URETRA, NO ESPECIFICADO",
"TRASTORNO DE LOS DISCOS INTERVERTEBRALES, NO ESPECIFICADO",
"TRASTORNO DE PANICO (ANSIEDAD PAROXISTICA EPISODICA)",
"TRASTORNO DE SOMATIZACION",
"TRASTORNO DEL SISTEMA URINARIO, NO ESPECIFICADO",
"TRASTORNO DEPRESIVO RECURRENTE, EPISODIO DEPRESIVO GRAVE PRESENTE SIN SINTOMAS PSICÃ“TICOS",
"TRASTORNO DEPRESIVO RECURRENTE, EPISODIO DEPRESIVO GRAVE PRESENTE, CON SINTOMAS PSICÃ“TICOS",
"TRASTORNO DISOCIATIVO (DE CONVERSION), NO ESPECIFICADO",
"TRASTORNO ESPECIFICO DE LA PRONUNCIACION",
"TRASTORNO FUNCIONAL INTESTINAL, NO ESPECIFICADO",
"TRASTORNO HEMORRÃGICO DEBIDO A ANTICOAGULANTES CIRCULANTES",
"TRASTORNO MIXTO DE ANSIEDAD Y DEPRESION",
"TRASTORNO PSICOTICO AGUDO Y TRANSITORIO, NO ESPECIFICADO",
"TRASTORNO QUE AFECTA AL MECANISMO DE LA INMUNIDAD, NO ESPECIFICADO",
"TRASTORNO SINOVIAL Y TENDINOSO, NO ESPECIFICADO",
"TRASTORNO SUDORIPARO APOCRINO, NO ESPECIFICADO",
"TRASTORNO VASCULAR AGUDO DE LOS INTESTINOS",
"TRASTORNO VASCULAR DEL INTESTINO, NO ESPECIFICADO",
"TRASTORNO VENOSO, NO ESPECIFICADO",
"TRASTORNOS DE DISCO LUMBAR Y OTROS, CON RADICULOPATIA",
"TRASTORNOS DE LA VALVULA PULMONAR EN ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"TRASTORNOS DEL HIGADO EN OTRAS ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"TRASTORNOS DEL METABOLISMO DE LOS ÃCIDOS GRASOS",
"TRASTORNOS DEL NERVIO VAGO",
"TRASTORNOS INFLAMATORIOS DE LA MAMA",
"TRASTORNOS RESPIRATORIOS EN OTRAS ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"TRASTORNOS SISTEMICOS DEL TEJIDO CONJUNTIVO EN OTRAS ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"TRAUMATISMO CEREBRAL DIFUSO",
"TRAUMATISMO CEREBRAL FOCAL",
"TRAUMATISMO DE LA CABEZA, NO ESPECIFICADO",
"TRAUMATISMO DE LA PIERNA, NO ESPECIFICADO",
"TRAUMATISMO DE OTROS Ã“RGANOS INTRAABDOMINALES",
"TRAUMATISMO DE OTROS VASOS SANGUINEOS DEL TORAX",
"TRAUMATISMO DEL OJO Y DE LA ORBITA, NO ESPECIFICADO",
"TRAUMATISMO DEL PIE Y DEL TOBILLO, NO ESPECIFICADO",
"TRAUMATISMO DEL TENDON Y MUSCULO EXTENSOR DE OTRO(S) DEDO(S) A NIVEL DE LA MUÃ‘ECA Y DE MANO",
"TRAUMATISMO DEL TORAX, NO ESPECIFICADO",
"TRAUMATISMO NO ESPECIFICADO DE LA CADERA Y DEL MUSLO",
"TRAUMATISMO POR APLASTAMIENTO DE LA RODILLA",
"TRAUMATISMO SUPERFICIAL DE LA CABEZA, PARTE NO ESPECIFICADA",
"TRAUMATISMO SUPERFICIAL DE LA NARIZ",
"TRAUMATISMO SUPERFICIAL DE OTRAS PARTES DE LA CABEZA",
"TRAUMATISMO SUPERFICIAL DE OTRAS PARTES Y DE LAS NO ESPECIFICADAS DEL TORAX",
"TRAUMATISMO SUPERFICIAL DEL ABDOMEN, DE LA REGION LUMBOSACRA Y DE LA PELVIS, PARTE NO ESPECIF.",
"TRAUMATISMO SUPERFICIAL DEL CUERO CABELLUDO",
"TRAUMATISMO SUPERFICIAL DEL LABIO Y DE LA CAVIDAD BUCAL",
"TRAUMATISMO, NO ESPECIFICADO",
"TRAUMATISMOS MULTIPLES DE LA CABEZA",
"TRAUMATISMOS MULTIPLES DE LA PIERNA",
"TRAUMATISMOS MULTIPLES, NO ESPECIFICADOS",
"TRAUMATISMOS SUPERFICIALES MULTIPLES DE LA CABEZA",
"TRAUMATISMOS SUPERFICIALES MULTIPLES DEL HOMBRO Y DEL BRAZO",
"TRAUMATISMOS SUPERFICIALES MULTIPLES, NO ESPECIFICADOS",
"TRAUMATISMOS SUPERFICIALES QUE AFECTAN LA CABEZA CON EL CUELLO",
"TROMBOCITOPENIA NO ESPECIFICADA",
"TROMBOFLEBITIS MIGRATORIA",
"TUBERCULOSIS DE PULMÃ“N, SIN EXAMEN BACTERIOLÃ“GICO E HISTOLÃ“GICO",
"TUMOR BENIGNO DE OTRAS PARTES ESPECIFICADAS DEL SISTEMA NERVIOSO CENTRAL",
"TUMOR BENIGNO DEL COLON, PARTE NO ESPECIFICADA",
"TUMOR BENIGNO DEL ESTOMAGO",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE Ã“RGANOS DIGESTIVOS, SITIO NO ESPECIFICADO",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE HÃGADO, DE VESÃCULA BILIAR Y CONDUCTO BILIAR",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE LA GLÃNDULA HIPÃ“FISIS",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE LA MEDULA ESPINAL",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE LA PELVIS RENAL",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE LAS MENINGES CEREBRALES",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE OTROS Ã“RGANOS DIGESTIVOS ESPECIFICADOS",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE OTROS Ã“RGANOS GENITALES FEMENINOS",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DE OTROS SITIOS ESPECIFICADOS",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DEL ENCÃ‰FALO, PARTE NO ESPECIFICADA",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DEL ESTOMAGO",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DEL MEDIASTINO",
"TUMOR DE COMPORTAMIENTO INCIERTO O DESCONOCIDO DEL RETROPERITONEO",
"TUMOR MALIGNO DE LA CABEZA DEL PÃNCREAS",
"TUMOR MALIGNO DE LA COLA DEL PÃNCREAS",
"TUMOR MALIGNO DE LA MAMA, PARTE NO ESPECIFICADA",
"TUMOR MALIGNO DE LA PLEURA",
"TUMOR MALIGNO DE LA VESÃCULA BILIAR",
"TUMOR MALIGNO DEL COLON, PARTE NO ESPECIFICADA",
"TUMOR MALIGNO DEL CUERPO DEL ESTOMAGO",
"TUMOR MALIGNO DEL ESÃ“FAGO, PORCIÃ“N TORÃCICA",
"TUMOR MALIGNO DEL ESTOMAGO, PARTE NO ESPECIFICADA",
"TUMOR MALIGNO DEL L+OBULO FRONTAL",
"TUMOR MALIGNO DEL RECTO",
"ULCERA CRONICA DE LA PIEL, NO CLASIFICADA EN OTRA PARTE",
"ULCERA DE MIEMBRO INFERIOR, NO CLASIFICADA EN OTRA PARTE",
"ULCERA DUODENAL AGUDA CON HEMORRAGIA",
"ULCERA DUODENAL AGUDA CON PERFORACION",
"ULCERA GASTRICA AGUDA CON PERFORACION",
"ULCERA GASTRICA CRONICA O NO ESPECIFICADA, CON HEMORRAGIA",
"ULCERA GASTRICA CRONICA SIN HEMORRAGIA NI PERFORACION",
"ULCERACION E INFLAMACION VULVOVAGINAL EN OTRAS ENFERMEDADES CLASIFICADAS EN OTROS CAPITULOS",
"URETRITIS EN ENFERMEDADES CLASIFICADAS EN OTRA PARTE",
"UROPATIA ASOCIADA CON REFLUJO VESICOURETERAL",
"URTICARIA ALERGICA",
"USO NOCIVO DE MULTIPLES DROGAS Y DE OTRAS SUSTANCIAS PSICOACTIVAS (ABUSO)",
"USO NOCIVO DE SEDANTES O HIPNÃ“TICOS (ABUSO)",
"VARICES EN OTROS SITIOS ESPECIFICADOS",
"VARICES ESCROTALES",
"VARICES ESOFAGICAS CON HEMORRAGIA",
"VENAS VARICOSAS DE LOS MIEMBROS INFERIORES CON INFLAMACION",
"VENAS VARICOSAS DE LOS MIEMBROS INFERIORES CON ULCERA E INFLAMACION",
"VENAS VARICOSAS DE LOS MIEMBROS INFERIORES CON ULCERA",
"VENAS VARICOSAS DE LOS MIEMBROS INFERIORES SIN ULCERA NI INFLAMACION",
"VERTIGO DE ORIGEN CENTRAL",
"VERTIGO PAROXISTICO BENIGNO",
"VOMITOS ASOCIADOS CON OTRAS ALTERACIONES PSICOLOGICAS",
])

In [ ]:
X[:,26] = le_diag.transform(X[:,26])

Asignación de la variable dependiente a predecir (categoría), al vector y

In [ ]:
X.shape

 <h1 id="Selección de atributos">Selección de atributos</h1>
<p>
Se identificará qué parámetros son mas útiles para la predicción,con el fin de reducir la dimensionalidad de la base de datos utilizada para la predicción de categoría. El objetivo de reducir la dimensionalidad de la base de datos es agilizar los procesos de entrenamiento y predicción de la categoria de pacientes.
</p>

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2
X = SelectKBest(chi2, k=10).fit_transform(X, y)
X.shape

 <h1 id="Normalización de datos">Normalización de datos</h1>
<p>
Para aplicar el paquete de arboles de desición, los datos deben estar en una escala similar, es por ello que optamos por normalizarlos para que los valores estén en un rango entre -2 y 2. Esta medida no altera los resultados aunque si faborece a la eficiencia de los algoritmos
</p>

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X)
X[0:5]

 <h1 id="Configurando algoritmos">Configurando algoritmos</h1>
<p>
En esta sección se definen parámetros necesarios para la correcta aplicación de los algoritmos a implementar, además de seccionar el conjunto de datos en datos de prueba(30%)y de entrenamiento (70%). Los parámetros escogidos pueden ser modificados con el fin de obtener resultados diferentes
</p>

Carga de paquetes

In [ ]:
from sklearn.model_selection import train_test_split

Segmentación del conjunto de datos

In [ ]:
X_trainset, X_testset, y_trainset, y_testset = train_test_split(X, y, test_size=0.3, random_state=3)

Resampling del conjunto de datos

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.ensemble import BalancedBaggingClassifier
from collections import Counter
 
os_us = SMOTETomek(sampling_strategy='auto',random_state=None,smote=None,tomek=None,n_jobs=None,)
X_trainset_res_ST1, y_trainset_res_ST1 = os_us.fit_sample(X_trainset, y_trainset)
 
print ("Distribution before resampling {}".format(Counter(y_trainset)))
print ("Distribution labels after resampling {}".format(Counter(y_trainset_res_ST1)))

Árbol de desición

In [ ]:
from sklearn.tree import DecisionTreeClassifier
DT= DecisionTreeClassifier(criterion="entropy", max_depth = 4)
DT.fit(X_trainset,y_trainset)
yhat_1 = DT.predict(X_testset)
yhat_prob_1=DT.predict_proba(X_testset)
DT_Acc=round(metrics.accuracy_score(y_testset, yhat_1),4)
#DT_Jcc=round(jaccard_similarity_score(y_testset, yhat_1),4)
DT_lgl=round(log_loss(y_testset, yhat_prob_1),4)
DT_F1=f1_score(y_testset, yhat_1, average='weighted') 

In [ ]:
DT_GS = DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='entropy',
                       max_depth=19, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0,
                       random_state=42, splitter='best')
DT_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
DT_parameters = [{'criterion': ['entropy', 'gini'], 'max_depth': np.arange(3, 21)},{'min_samples_leaf': [5, 10, 20, 50, 100]}]
DT_GS = GridSearchCV(DecisionTreeClassifier(random_state=42), DT_parameters, verbose=1, cv=5, scoring='balanced_accuracy')
DT_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
DT_GS.best_estimator_

In [ ]:
yhat_1_GS= DT_GS.predict(X_testset)
yhat_prob_1_GS=DT_GS.predict_proba(X_testset)
DT_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_1_GS),4)
#DT_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_1_GS),4)
DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS),4)
DT_F1_GS=round(f1_score(y_testset, yhat_1_GS, average='weighted'),4) 

In [ ]:
yhat_1_GS_ST= DT_GS_ST.predict(X_testset)
yhat_prob_1_GS=DT_GS.predict_proba(X_testset)
DT_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_1_GS_ST),4)
#DT_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_1_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
DT_F1_GS_ST=round(f1_score(y_testset, yhat_1_GS_ST, average='weighted'),4) 

In [ ]:
resultados_DT = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Árboles de decisión':[DT_Acc,DT_F1],
             'Grid Search':[DT_Acc_GS,DT_F1_GS],
                'GS+SMOTE-Tomek':[DT_Acc_GS_ST,DT_F1_GS_ST]}
Tabla_resultados_DT=pd.DataFrame(resultados_DT)
print(Tabla_resultados_DT)

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance

In [ ]:
perm = PermutationImportance(DT, random_state=1).fit(X, y)
eli5.show_weights(perm, feature_names =X.columns.tolist())

In [ ]:
import shap
ex = shap.TreeExplainer(DT)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(DT.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Bosque de desición

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF= RandomForestClassifier(max_depth=2, random_state=0)
RF.fit(X_trainset,y_trainset)
yhat_2 = RF.predict(X_testset)
yhat_prob_2=RF.predict_proba(X_testset)
RF_Acc=round(metrics.accuracy_score(y_testset, yhat_2),4)
#RF_Jcc=round(jaccard_similarity_score(y_testset, yhat_2),4)
RF_lgl=round(log_loss(y_testset, yhat_prob_2),4)
RF_F1=f1_score(y_testset, yhat_2, average='weighted') 

In [ ]:
RF_GS =RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=20, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=400,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
RF_GS.fit(X_trainset,y_trainset)
DT_GS_ST=DT_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
RF_parameters ={ 'n_estimators': [100,200,300,400,500],'criterion': ['entropy', 'gini'], 'max_depth': np.arange(3, 21)},{'min_samples_leaf': [5, 10, 20, 50, 100]}
RF_GS = GridSearchCV(estimator=RF,param_grid=RF_parameters, cv= 3,scoring="accuracy")
RF_GS.fit(X_trainset,y_trainset)
RF_GS_ST=RF_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
RF_GS.best_estimator_

In [ ]:
yhat_2_GS= RF_GS.predict(X_testset)
yhat_prob_2_GS=RF_GS.predict_proba(X_testset)
RF_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_2_GS),4)
#RF_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_2_GS),4)
#RF_lgl_GS=round(log_loss(y_testset, yhat_prob_2_GS),4)
RF_F1_GS=round(f1_score(y_testset, yhat_2_GS, average='weighted'),4) 

In [ ]:
yhat_2_GS_ST= DT_GS_ST.predict(X_testset)
yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
RF_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_2_GS_ST),4)
#RF_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_2_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
RF_F1_GS_ST=round(f1_score(y_testset, yhat_2_GS_ST, average='weighted'),4) 

In [ ]:
resultados_RF = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Bosque de decisión':[RF_Acc,RF_F1],
             'Grid Search':[RF_Acc_GS,RF_F1_GS],
                'GS+SMOTE-Tomek':[RF_Acc_GS_ST,RF_F1_GS_ST]}
Tabla_resultados_RF=pd.DataFrame(resultados_RF)
print(Tabla_resultados_RF)

In [ ]:
import shap
ex = shap.TreeExplainer(RF)
shap_values = ex.shap_values(X)
shap.summary_plot(shap_values, X)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(RF.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Regresión logística (multi-class)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
LR = LogisticRegression(C=0.01, solver='newton-cg',penalty='l2',multi_class='ovr').fit(X_trainset,y_trainset)
yhat_3= LR.predict(X_testset)
yhat_prob_3=LR.predict_proba(X_testset)
LR_Acc=round(metrics.accuracy_score(y_testset, yhat_3),4)
#LR_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
LR_lgl=round(log_loss(y_testset, yhat_prob_3),4)
LR_F1=f1_score(y_testset, yhat_3, average='weighted') 

In [ ]:
LR_GS = LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='ovr', n_jobs=None, penalty='l2',
                   random_state=None, solver='newton-cg', tol=0.0001, verbose=0,
                   warm_start=False)
LR_GS.fit(X_trainset,y_trainset)
LR_GS_ST=LR_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
LR_parameters ={'C':(0.1,1,10), 'penalty':['l1','l2','elasticnet']}
LR_GS = GridSearchCV(estimator=LR,param_grid=LR_parameters, verbose=True, n_jobs=-1,cv=3)
LR_GS.fit(X_trainset,y_trainset)
LR_GS_ST=LR_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
LR_GS.best_estimator_

In [ ]:
yhat_3_GS= LR_GS.predict(X_testset)
yhat_prob_3_GS=LR_GS.predict_proba(X_testset)
LR_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_3_GS),4)
#LR_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_3_GS),4)
LR_lgl_GS=round(log_loss(y_testset, yhat_prob_3_GS),4)
LR_F1_GS=round(f1_score(y_testset, yhat_3_GS, average='weighted'),4) 

In [ ]:
yhat_3_GS= LR_GS.predict(X_testset)
yhat_prob_3_GS=LR_GS.predict_proba(X_testset)
LR_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_3_GS),4)
#LR_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_3_GS),4)
LR_lgl_GS=round(log_loss(y_testset, yhat_prob_3_GS),4)
LR_F1_GS=round(f1_score(y_testset, yhat_3_GS, average='weighted'),4) 

In [ ]:
yhat_3_GS_ST= LR_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
LR_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_3_GS_ST),4)
#LR_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_3_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
LR_F1_GS_ST=round(f1_score(y_testset, yhat_3_GS_ST, average='weighted'),4) 

In [ ]:
resultados_LR = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Regresión Logística':[LR_Acc,LR_F1],
             'Grid Search':[LR_Acc_GS,LR_F1_GS],
                 'GS+SMOTE-Tomek':[LR_Acc_GS_ST,LR_F1_GS_ST]}
Tabla_resultados_LR=pd.DataFrame(resultados_LR)
print(Tabla_resultados_LR)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(LR.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

Redes Neuronales

In [ ]:
from sklearn.neural_network import MLPClassifier
NN = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1).fit(X_trainset,y_trainset)
yhat_4 = NN.predict(X_testset)
yhat_prob_4=NN.predict_proba(X_testset)
NN_Acc=round(metrics.accuracy_score(y_testset, yhat_4),4)
#NN_Jcc=round(jaccard_similarity_score(y_testset, yhat_3),4)
NN_lgl=round(log_loss(y_testset, yhat_prob_4),4)
NN_F1=f1_score(y_testset, yhat_4, average='weighted') 

In [ ]:
NN_GS = MLPClassifier(activation='tanh', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(50, 100, 50), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=1, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)
NN_GS.fit(X_trainset,y_trainset)
NN_GS_ST=NN_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
NN_parameters = {'hidden_layer_sizes': [(50,50,50), (50,100,50), (100,)],'activation': ['tanh', 'relu'],
    'solver': ['sgd', 'adam'],'alpha': [0.0001, 0.05],'learning_rate': ['constant','adaptive']}

NN_GS = GridSearchCV(estimator=NN,param_grid=NN_parameters, cv= 3, verbose=True,scoring="accuracy")
NN_GS.fit(X_trainset,y_trainset)
NN_GS_ST=NN_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
NN_GS.best_estimator_

In [ ]:
yhat_4_GS= NN_GS.predict(X_testset)
#yhat_prob_4_GS=NN_GS.predict_proba(X_testset)
NN_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_4_GS),4)
#NN_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_4_GS),4)
#NN_lgl_GS=round(log_loss(y_testset, yhat_prob_4_GS),4)
NN_F1_GS=round(f1_score(y_testset, yhat_4_GS, average='weighted'),4) 

In [ ]:
yhat_4_GS_ST= NN_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
NN_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_4_GS_ST),4)
#NN_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_4_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
NN_F1_GS_ST=round(f1_score(y_testset, yhat_4_GS_ST, average='weighted'),4) 

In [ ]:
resultados_NN = {'índices de rendimiento':['Accuracy','F1-Score'],
             'Red Neuronal':[NN_Acc,NN_F1],
             'Grid Search':[NN_Acc_GS,NN_F1_GS],
                'GS+SMOTE-Tomek':[NN_Acc_GS_ST,NN_F1_GS_ST]}
Tabla_resultados_NN=pd.DataFrame(resultados_NN)
print(Tabla_resultados_NN)

In [ ]:
data_DF=pd.DataFrame(X, columns=df_hosp.columns)
data_DF

In [ ]:
import eli5
from eli5.sklearn import PermutationImportance
perm = PermutationImportance(NN, random_state=1).fit(data_DF, y)
eli5.show_weights(perm, feature_names =data_DF.columns.tolist())

In [ ]:
# Kerner Explainer
import shap
explainer = shap.KernelExplainer(NN.predict_proba,data_DF[:100])
shap_values = explainer.shap_values(data_DF[:100])
shap.summary_plot(shap_values, data_DF[:100])

Support Vector Machine

In [ ]:
from sklearn import svm
SVM = svm.SVC(kernel='rbf',decision_function_shape='ovo', probability=True)
SVM.fit(X_trainset, y_trainset) 
yhat_5 = SVM.predict(X_testset)
yhat_prob_5=SVM.predict_proba(X_testset)
SVM_Acc=round(metrics.accuracy_score(y_testset, yhat_5),4)
#SVM_Jcc=round(jaccard_similarity_score(y_testset, yhat_5),4)
SVM_lgl=round(log_loss(y_testset, yhat_prob_5),4)
SVM_F1=f1_score(y_testset, yhat_5, average='weighted') 

In [ ]:
SVM_parameters ={'C': [0.1,1, 10, 100], 'gamma': [1,0.1,0.01,0.001],'kernel': ['rbf', 'poly', 'sigmoid']}
SVM_GS = GridSearchCV(estimator=SVM,param_grid=SVM_parameters, cv= 3, verbose=True, n_jobs=-1)
SVM_GS.fit(X_trainset,y_trainset)
SVM_GS_ST=SVM_GS.fit(X_trainset_res_ST1, y_trainset_res_ST1)

In [ ]:
SVM_GS.best_estimator_

In [ ]:
yhat_5_GS= SVM_GS.predict(X_testset)
yhat_prob_5_GS=SVM_GS.predict_proba(X_testset)
SVM_Acc_GS=round(metrics.accuracy_score(y_testset, yhat_5_GS),4)
#SVM_Jcc_GS=round(jaccard_similarity_score(y_testset, yhat_5_GS),4)
SVM_lgl_GS=round(log_loss(y_testset, yhat_prob_5_GS),4)
SVM_F1_GS=round(f1_score(y_testset, yhat_5_GS, average='weighted'),4) 

In [ ]:
yhat_5_GS_ST= NN_GS_ST.predict(X_testset)
#yhat_prob_2_GS=DT_GS.predict_proba(X_testset)
SVM_Acc_GS_ST=round(metrics.accuracy_score(y_testset, yhat_5_GS_ST),4)
SVM_Jcc_GS_ST=round(jaccard_similarity_score(y_testset, yhat_5_GS_ST),4)
#DT_lgl_GS=round(log_loss(y_testset, yhat_prob_1_GS_ST),4)
SVM_F1_GS_ST=round(f1_score(y_testset, yhat_5_GS_ST, average='weighted'),4) 

In [ ]:
resultados_SVM = {'índices de rendimiento':['Accuracy','Jaccard','F1-Score'],
             'Red Neuronal':[SVM_Acc,SVM_Jcc,SVM_F1],
             'Grid Search':[SVM_Acc_GS,SVM_Jcc_GS,SVM_F1_GS],
             'GS+SMOTE-Tomek':[SVM_Acc_GS_ST,SVM_Jcc_GS_ST,SVM_F1_GS_ST]}
Tabla_resultados_SVM=pd.DataFrame(resultados_SVM)
print(Tabla_resultados_SVM)

In [ ]:
# Kerner Explainer
explainer = shap.KernelExplainer(SVM.predict_proba,X[:100])
shap_values = explainer.shap_values(X[:100])
shap.summary_plot(shap_values, X[:100])

In [ ]:
resultados = {'Algoritmos de clasificación':['Árboles de decisión','AD+GS','Bosques de decisión','BD+GS','Regresión Logística','RL+GS','Red Neuronal','RN+GS'],
             'Accuracy':[DT_Acc,DT_Acc_GS,RF_Acc,RF_Acc_GS,LR_Acc,LR_Acc_GS,NN_Acc,NN_Acc_GS],
                     'F1-Score':[DT_F1,DT_F1_GS,RF_F1,RF_F1_GS,LR_F1,LR_F1_GS,NN_F1,NN_F1_GS]}
Tabla_resultados=pd.DataFrame(resultados)
print(Tabla_resultados)


 <h1 id="Matrices de confusión">Matrices de confusión</h1>
<p>
En esta sección se construyen matrices de confusión para evaluar la eficacia de cada algoritmo
</p>

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import itertools
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Matriz de confusión',
                          cmap=plt.cm.Blues):
    """
    Esta función muestra y dibuja la matriz de confusión.
    La normalización se puede aplicar estableciendo el valor `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Matriz de confusión normalizada")
    else:
        print('Matriz de confusión sin normalización')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('Etiqueta Real')
    plt.xlabel('Etiqueta Predicha')


Árbol de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_1, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión árbol de decisión')

Bosque de decisión

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_2_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión bosque de decisión + ST')

Regresión Logística

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_3_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión regresión logística +ST')

Red neuronal

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_4_GS_ST, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión red neuronal +ST')

Support Vector Machine

In [ ]:
cnf_matrix = confusion_matrix(y_testset, yhat_5, labels=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'])
np.set_printoptions(precision=2)
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=['A.P.S','DOMICILIO','HOSPITALIZAR','OTRO','OTRO ESTABLECIMIENTO'],normalize= False,  title='Matriz de confusión SVM')